<a href="https://colab.research.google.com/github/PDNow-Research/PDNow/blob/main/HandPD/SVM_Exact_Replication_Experiment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Data Science
import re
import csv
import json
import itertools
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# General
import os
import sys
import time
import math
import random
from datetime import date
import warnings
current_date = date.today()
warnings.filterwarnings("ignore")

# SVM
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, average_precision_score, classification_report

In [12]:
#NewSpiral is from the new HandPD dataset. The new one also comes with images and such, but we are using the data that has the exact same features from old HandPd dataset, except also includes patient ID.
spiral_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewSpiral.csv')

In [13]:
spiral_df.shape

(264, 16)

In [14]:
spiral_df['CLASS_TYPE'].value_counts()

1    140
2    124
Name: CLASS_TYPE, dtype: int64

In [9]:
meander_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewMeander.csv')

In [10]:
meander_df.columns

Index(['_ID_EXAM', 'IMAGE_NAME', 'ID_PATIENT', 'CLASS_TYPE', 'GENDER',
       'RIGH/LEFT-HANDED', 'AGE', 'RMS', 'MAX_BETWEEN_ET_HT',
       'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT',
       'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT'],
      dtype='object')

In [15]:
spiral_df['ID_PATIENT'].unique().tolist()

[59,
 76,
 301,
 102,
 305,
 104,
 127,
 297,
 299,
 98,
 78,
 80,
 86,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 143,
 31,
 38,
 47,
 53,
 235,
 253,
 261,
 268,
 273,
 281,
 247,
 149,
 157,
 165,
 186,
 192,
 196,
 207,
 218,
 203,
 224,
 153,
 176,
 181,
 230,
 138,
 187]

In [16]:
spiral_df[spiral_df['ID_PATIENT'] == 31]

,_ID_EXAM,IMAGE_NAME,ID_PATIENT,CLASS_TYPE,GENDER,RIGH/LEFT-HANDED,AGE,RMS,MAX_BETWEEN_ET_HT,MIN_BETWEEN_ET_HT,STD_DEVIATION_ET_HT,MRT,MAX_HT,MIN_HT,STD_HT,CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT
141,P2,mea1-P2.jpg,31,2,F,R,59,3726.488067,5893.347008,37785.94529,0.306602,24.403343,179.554888,0.008474,1739.851476,0.267742
164,P26,mea1-P26.jpg,31,2,F,L,59,4587.166408,6561.231877,37033.96196,0.018378,22.328549,184.806500,0.057158,1388.315960,0.266871
172,P2,mea2-P2.jpg,31,2,F,R,59,3469.841548,5282.395889,33582.16916,0.000129,18.103416,170.531514,0.045752,1130.591703,0.204545
195,P26,mea2-P26.jpg,31,2,F,L,59,6210.596735,7365.102809,37813.01828,0.002359,20.958406,160.395258,0.024497,1281.094994,0.128205
203,P2,mea3-P2.jpg,31,2,F,R,59,5066.989293,7463.980609,35307.01259,0.000000,22.643151,176.392744,0.008650,1614.293957,0.246711
226,P26,mea3-P26.jpg,31,2,F,L,59,6544.571924,8612.729092,36154.08597,0.013198,27.237956,180.306815,0.003577,1761.998890,0.243421
234,P2,mea4-P2.jpg,31,2,F,R,59,3748.920825,6607.203671,36244.36135,0.001141,24.606020,179.208863,0.028543,1782.895068,0.263158
257,P26,mea4-P26.jpg,31,2,F,L,59,4921.267695,6948.380564,35304.08954,0.017636,25.699685,182.482156,0.055527,1698.122391,0.235915


From the above, we can see that there are two patients with the ID of 31. So we're going to give one of them an ID of 32 instead, as that ID does not belong to any other.

In [17]:
spiral_df['ID_PATIENT'][spiral_df['_ID_EXAM'] == "P26"] = 32

In [18]:
spiral_df[spiral_df['ID_PATIENT'] == 32]

,_ID_EXAM,IMAGE_NAME,ID_PATIENT,CLASS_TYPE,GENDER,RIGH/LEFT-HANDED,AGE,RMS,MAX_BETWEEN_ET_HT,MIN_BETWEEN_ET_HT,STD_DEVIATION_ET_HT,MRT,MAX_HT,MIN_HT,STD_HT,CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT
164,P26,mea1-P26.jpg,32,2,F,L,59,4587.166408,6561.231877,37033.96196,0.018378,22.328549,184.806500,0.057158,1388.315960,0.266871
195,P26,mea2-P26.jpg,32,2,F,L,59,6210.596735,7365.102809,37813.01828,0.002359,20.958406,160.395258,0.024497,1281.094994,0.128205
226,P26,mea3-P26.jpg,32,2,F,L,59,6544.571924,8612.729092,36154.08597,0.013198,27.237956,180.306815,0.003577,1761.998890,0.243421
257,P26,mea4-P26.jpg,32,2,F,L,59,4921.267695,6948.380564,35304.08954,0.017636,25.699685,182.482156,0.055527,1698.122391,0.235915


##**Preprocessing**

###Normalization

fi' = (fi - avg)/std

In [19]:
# normalization with the paper's method (formula above)
def feature_normalization(df):
  df_dup = df.copy()

  df1 = df_dup[['RMS', 'MAX_BETWEEN_ET_HT',
       'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT',
       'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT']]

  df2 = df_dup[['_ID_EXAM', 'IMAGE_NAME', 'ID_PATIENT', 'CLASS_TYPE', 'GENDER',
       'RIGH/LEFT-HANDED', 'AGE']]
  avg_dev = df1.mad(axis = 0)
  std_dev = df1.std(axis = 0)

  df1 = df1.sub(avg_dev)
  df1 = df1.divide(std_dev)

  return df2, df1

In [20]:
# normalizing spiral_df
spiral_df = pd.concat(feature_normalization(spiral_df), axis = 1)

In [21]:
# normalizing X_meander
meander_df =  pd.concat(feature_normalization(meander_df), axis = 1)

###Data Splitting
"The first one [experiment] uses 75% of the dataset for training purposes and the remaining 25% for the classification phase. However, instead of partitioning the dataset randomly, we created four subsets in order to guarantee that each individual will be represented in the dataset with its 3 spirals/meanders, with the remaining one being used for classification purposes. In this experiment, the spiral- and meander-based datasets are used individually."

In [200]:
def split_data(df, dfarr):
  df_IDs = df['ID_PATIENT'].unique().tolist()

  for x in df_IDs:
    for i in range(len(dfarr)):
      row = df[df['ID_PATIENT'] == x].iloc[i]
      dfarr[i] = dfarr[i].append(row)

  return dfarr

In [211]:
def represented_train_test_split(df):

  test = pd.DataFrame(columns = df.columns)
  train = pd.DataFrame(columns = df.columns)

  dfarr = [] # creating list of lists with 4 lists
  for i in range(4):
    dfarr.append(pd.DataFrame(columns = df.columns))

  # Some patients may have multiple test IDs so we must base this on Patient ID, which can be same for PD and Ctrl groups. So we split them.
  ctrl_df = df.query('CLASS_TYPE == 1')
  pd_df = df.query('CLASS_TYPE == 2')

  # we can append very easily because each of the IDs are different in ctrl_df and pd_df, since we split from X_spiral, which has all unique IDs
  dfarr = split_data(ctrl_df, dfarr)
  dfarr = split_data(pd_df, dfarr)

  test = dfarr[0]
  test = test.reset_index(drop=True)


  train = pd.concat(dfarr[0:3]) # excludes 0
  train = train.reset_index(drop=True)

  return test, train

In [212]:
spiral_test, spiral_train = represented_train_test_split(spiral_df)

In [213]:
spiral_test.shape

(66, 16)

In [214]:
spiral_train.shape

(198, 16)

In [26]:
spiral_test.shape

(66, 16)

In [215]:
spiral_test['ID_PATIENT'].value_counts()

5      2
23     2
14     1
17     1
38     1
      ..
53     1
181    1
305    1
176    1
1      1
Name: ID_PATIENT, Length: 64, dtype: int64

In [216]:
spiral_train['ID_PATIENT'].value_counts()

5      6
23     6
14     3
17     3
38     3
      ..
53     3
181    3
305    3
176    3
1      3
Name: ID_PATIENT, Length: 64, dtype: int64

As can be seen, we end up with only 2 IDs that are duplicated in PD and control patients: 5 and 23.

###Train-Test-Split into X and y

In [29]:
spiral_train.columns

Index(['_ID_EXAM', 'IMAGE_NAME', 'ID_PATIENT', 'CLASS_TYPE', 'GENDER',
       'RIGH/LEFT-HANDED', 'AGE', 'RMS', 'MAX_BETWEEN_ET_HT',
       'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT',
       'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT'],
      dtype='object')

In [30]:
# creating our final training dataset with spiral drawings
X_spiral_train = spiral_train[['RMS', 'MAX_BETWEEN_ET_HT', 'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT', 'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT']]
y_spiral_train = spiral_train['CLASS_TYPE']

y_spiral_train = pd.DataFrame(y_spiral_train)

In [31]:
# creating our final testing dataset with spiral drawings
X_spiral_test = spiral_test[['RMS', 'MAX_BETWEEN_ET_HT', 'MIN_BETWEEN_ET_HT', 'STD_DEVIATION_ET_HT', 'MRT', 'MAX_HT', 'MIN_HT', 'STD_HT', 'CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT']]
y_spiral_test = spiral_test['CLASS_TYPE']

y_spiral_test = pd.DataFrame(y_spiral_test)

###Quick Check

In [32]:
y_spiral_train.value_counts()

CLASS_TYPE
1             105
2              93
dtype: int64

In [33]:
y_spiral_test.value_counts()

CLASS_TYPE
1             35
2             31
dtype: int64

##**SVM Implementation**

###**Training**

In [34]:
clf = SVC(kernel = 'rbf', probability = True, class_weight = 'balanced')
clf.fit(X_spiral_train, y_spiral_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

To note, it may be useful to look at the weights for this model (see which features are most valuable), but these are only relevant for a linear kernel. RBF kernel does not have relevant/interpretable weights. And of course, linear kernel is not very helpful and good in predicting (51% accuracy). The features are not linearly separable.

In [151]:
y_spiral_pred = clf.predict(X_spiral_test)
y_spiral_proba = clf.predict_proba(X_spiral_test)

In [81]:
y_spiral_pred = pd.Series(y_spiral_pred)

In [152]:
y_spiral_proba

array([[0.99895935, 0.00104065],
       [0.76576917, 0.23423083],
       [0.99566507, 0.00433493],
       [0.5826891 , 0.4173109 ],
       [0.70784483, 0.29215517],
       [0.33551129, 0.66448871],
       [0.38530444, 0.61469556],
       [0.33688005, 0.66311995],
       [0.3354091 , 0.6645909 ],
       [0.64445386, 0.35554614],
       [0.27663295, 0.72336705],
       [0.20930562, 0.79069438],
       [0.33327959, 0.66672041],
       [0.2936485 , 0.7063515 ],
       [0.35876601, 0.64123399],
       [0.48083895, 0.51916105],
       [0.29585993, 0.70414007],
       [0.63633983, 0.36366017],
       [0.56735444, 0.43264556],
       [0.73034009, 0.26965991],
       [0.97833417, 0.02166583],
       [0.33395153, 0.66604847],
       [0.74778493, 0.25221507],
       [0.48343181, 0.51656819],
       [0.37038538, 0.62961462],
       [0.55388041, 0.44611959],
       [0.68226635, 0.31773365],
       [0.99048319, 0.00951681],
       [0.94060436, 0.05939564],
       [0.99627507, 0.00372493],
       [0.

In [159]:
len([i for i in y_spiral_proba if i < 0.2775])


array([1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2,
       1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

##**Results**

#### Deviance of probabilites from 100/0 depending on class

In [153]:
# flattening y_spiral_proba & removing the second percentage we don't need

# y_spiral_proba = [x for list in y_spiral_proba for x in list]
y_spiral_proba = y_spiral_proba[0::2] # takes every other element starting from the first element

In [154]:
y_spiral_proba

[0.9989593501926906,
 0.7657691737400566,
 0.995665066877425,
 0.5826891008680899,
 0.7078448261262179,
 0.3355112910628663,
 0.38530444320491547,
 0.33688005174320407,
 0.3354091041216158,
 0.6444538555257089,
 0.2766329500044335,
 0.20930562430301605,
 0.3332795872446732,
 0.2936485010004472,
 0.3587660078503655,
 0.48083894728789106,
 0.2958599298149011,
 0.6363398349971425,
 0.5673544406535503,
 0.7303400885579483,
 0.9783341669979735,
 0.3339515312511905,
 0.7477849319442436,
 0.4834318107070946,
 0.37038537880353595,
 0.553880408680958,
 0.6822663549766973,
 0.9904831942662662,
 0.9406043596541583,
 0.9962750705502315,
 0.6790366841003395,
 0.9825117079137469,
 0.42631619591310893,
 0.43493055737245134,
 0.7792052757676815,
 0.4077495188636984,
 0.27266381465310235,
 0.3024502860402416,
 0.3843285848807891,
 0.4661192207979928,
 0.5647295335677937,
 0.39138430781865213,
 0.5647295335677937,
 0.39138430781865213,
 0.7481736458542284,
 0.2918745262165278,
 0.21346910488841814,
 0.3

In [155]:
len(y_spiral_proba)

66

In [160]:
# calculating standard dev 

proba1 = [i for i in y_spiral_proba if i < 0.5]
proba2 = [i for i in y_spiral_proba if i > 0.5]

In [164]:
print(proba2)

[0.9989593501926906, 0.7657691737400566, 0.995665066877425, 0.5826891008680899, 0.7078448261262179, 0.6444538555257089, 0.6363398349971425, 0.5673544406535503, 0.7303400885579483, 0.9783341669979735, 0.7477849319442436, 0.553880408680958, 0.6822663549766973, 0.9904831942662662, 0.9406043596541583, 0.9962750705502315, 0.6790366841003395, 0.9825117079137469, 0.7792052757676815, 0.5647295335677937, 0.5647295335677937, 0.7481736458542284]


In [175]:
meanval =0

for i in proba1:
  meanval += (i)

meanval /= len(proba1)

meanval

0.33446681237722947

In [162]:
len(proba2)

22

#### Y Spiral Results

In [37]:
y_spiral_test = y_spiral_test['CLASS_TYPE'].astype(str).astype(int)

In [38]:
y_spiral_test.dtypes

dtype('int64')

In [39]:
clf.score(X_spiral_test, y_spiral_test)

0.6212121212121212

In [40]:
target_names = ['Control', 'PD']
results = classification_report(y_spiral_test, y_spiral_pred, target_names = target_names, output_dict=True)
results = pd.DataFrame(results).transpose()
conf_mat = confusion_matrix(y_spiral_test, y_spiral_pred)

In [41]:
results

,precision,recall,f1-score,support
Control,0.916667,0.314286,0.468085,35.000000
PD,0.555556,0.967742,0.705882,31.000000
accuracy,0.621212,0.621212,0.621212,0.621212
macro avg,0.736111,0.641014,0.586984,66.000000
weighted avg,0.747054,0.621212,0.579778,66.000000


In [42]:
conf_mat

array([[11, 24],
       [ 1, 30]])

In [43]:
TN, FP, FN, TP = conf_mat.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)

# Specificity or true negative rate
TNR = TN/(TN+FP) 

# Precision or positive predictive value
PPV = TP/(TP+FP)

# Negative predictive value
NPV = TN/(TN+FN)

# Fall out or false positive rate
FPR = FP/(FP+TN)

# False negative rate
FNR = FN/(TP+FN)

# False discovery rate
FDR = FP/(TP+FP)

print("TP: ", TP)
print("TN: ", TN)
print("FP: ", FP)
print("FN: ", FN)

print("Sensitivity: ", TPR)
print("Specificity: ", TNR)
print("NPV: ", NPV)
print("PPV: ", PPV)

TP:  30
TN:  11
FP:  24
FN:  1
Sensitivity:  0.967741935483871
Specificity:  0.3142857142857143
NPV:  0.9166666666666666
PPV:  0.5555555555555556


Can we somehow get the weights for the model to learn which features are considered most useful?